In [13]:
from mysql_config import get_mysql_connection, close_mysql_connection

connection = get_mysql_connection()

if connection:
    print("Conexión a MySQL exitosa!")
    close_mysql_connection(connection)
else:
    print("No se pudo conectar a MySQL.")

Host: 127.0.0.1, User: root, Password: Admin123, Database: tarea_viva, Port: 3306
Base de datos 'tarea_viva' seleccionada correctamente.
Conexión a MySQL exitosa!


In [14]:
from mysql_config import get_mysql_connection, close_mysql_connection

def crear_tarea(titulo, descripcion, archivos=None):
    try:
        with get_mysql_connection() as connection:
            if connection:
                with connection.cursor() as cursor:
                    sql = "INSERT INTO tareas (titulo, descripcion) VALUES (%s, %s)"
                    val = (titulo, descripcion)
                    cursor.execute(sql, val)
                    connection.commit()
                    tarea_id = cursor.lastrowid  # Obtener el ID de la tarea recién creada
                    print(f"Tarea '{titulo}' creada con éxito. ID: {tarea_id}")

                    if archivos:
                        for archivo in archivos:
                            agregar_archivo(tarea_id, archivo["nombre_archivo"], archivo["tipo_archivo"], archivo["ruta_archivo"])
            else:
                print("No se pudo conectar a MySQL.")
    except mysql.connector.Error as error:
        print(f"Error al crear la tarea: {error}")

def consultar_tareas():
    try:
        with get_mysql_connection() as connection:
            if connection:
                with connection.cursor() as cursor:
                    cursor.execute("SELECT * FROM tareas")
                    tareas = cursor.fetchall()
                    if tareas:
                        print("Lista de tareas:")
                        for tarea in tareas:
                            print(f"ID: {tarea[0]}, Título: {tarea[1]}, Descripción: {tarea[2]}, Estado: {tarea[4]}")
                            consultar_archivos(tarea[0])  # Consultar archivos relacionados a la tarea
                    else:
                        print("No hay tareas registradas.")
            else:
                print("No se pudo conectar a MySQL.")
    except mysql.connector.Error as error:
        print(f"Error al consultar las tareas: {error}")

def eliminar_tarea(id_tarea):
    connection = get_mysql_connection()
    if connection:
        cursor = connection.cursor()
        sql = "DELETE FROM tareas WHERE id = %s"
        val = (id_tarea,)
        cursor.execute(sql, val)
        connection.commit()
        print(f"Tarea con ID {id_tarea} eliminada con éxito.")
        close_mysql_connection(connection)
    else:
        print("No se pudo conectar a MySQL.")

In [15]:
# Crear una tarea
crear_tarea("Comprar leche", " 11111en el supermercado.")

# Consultar todas las tareas
consultar_tareas()

# Crear otra tarea
crear_tarea("Llamar al médico", "111111Pedir cita para el .")

# Consultar todas las tareas nuevamente
consultar_tareas()

# Eliminar la primera tarea (ID 1)
eliminar_tarea(1)

# Consultar todas las tareas después de eliminar
consultar_tareas()

Host: 127.0.0.1, User: root, Password: Admin123, Database: tarea_viva, Port: 3306
Base de datos 'tarea_viva' seleccionada correctamente.
Tarea 'Comprar leche' creada con éxito. ID: 7
Host: 127.0.0.1, User: root, Password: Admin123, Database: tarea_viva, Port: 3306
Base de datos 'tarea_viva' seleccionada correctamente.
Lista de tareas:
ID: 2, Título: Llamar al médico, Descripción: Pedir cita para el próximo mes., Estado: Pendiente
Conexión a MongoDB Atlas exitosa!
Consultando archivos para la tarea ID: 2
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2CBC71D0>
Tarea ID: 2, Nombre: Taller de Análisis.pdf, Tipo: Taller de Análisis, Ruta: C:/Users/Fernando/Downloads/Taller de Análisis.pdf
ID: 3, Título: Comprar leche, Descripción: Comprar leche descremada en el supermercado., Estado: Pendiente
Conexión a MongoDB Atlas exitosa!
Consultando archivos para la tarea ID: 3
ID: 4, Título: Comprar leche, Descripción: Comprar leche descremada en el supermercado., Estado: Pendiente
Co

In [ ]:
#conexión a MongoDB
from mongo_config import get_mongo_client, close_mongo_client

client = get_mongo_client()

if client:
    close_mongo_client(client)

Conexión a MongoDB Atlas exitosa!


In [5]:
from mongo_config import get_mongo_client, close_mongo_client
from mongo_models import Archivo

def agregar_archivo(tarea_id, nombre_archivo, tipo_archivo, ruta_archivo):
    try:
        client = get_mongo_client()
        if client:
            db = client.tareaviva_db
            collection = db.archivos
            archivo = Archivo(tarea_id, nombre_archivo, tipo_archivo, ruta_archivo)
            collection.insert_one(archivo.to_dict())
            print(f"Archivo '{nombre_archivo}' agregado con éxito.")
            close_mongo_client(client)
    except Exception as e:
        print(f"Error al agregar el archivo: {e}")

def consultar_archivos(tarea_id):
    try:
        client = get_mongo_client()
        if client:
            db = client.tareaviva_db
            collection = db.archivos
            print(f"Consultando archivos para la tarea ID: {tarea_id}")  # Mensaje de depuración
            archivos = collection.find({"tarea_id": tarea_id})
            for archivo in archivos:
                archivo_obj = Archivo.from_dict(archivo)
                print(f"Archivo encontrado: {archivo_obj}")  # Mensaje de depuración
                print(f"Tarea ID: {archivo_obj.tarea_id}, Nombre: {archivo_obj.nombre_archivo}, Tipo: {archivo_obj.tipo_archivo}, Ruta: {archivo_obj.ruta_archivo}")
            close_mongo_client(client)
        else:
            print("No se pudo conectar a MongoDB Atlas.")
    except Exception as e:
        print(f"Error al consultar los archivos: {e}")

In [ ]:
agregar_archivo(1, "imagen1.jpg", "imagen", "/ruta/a/imagen1.jpg")
consultar_archivos(1)

Conexión a MongoDB Atlas exitosa!
Archivo 'imagen1.jpg' agregado con éxito.
Conexión a MongoDB Atlas exitosa!
Tarea ID: 1, Nombre: imagen1.jpg, Tipo: imagen, Ruta: /ruta/a/imagen1.jpg
Tarea ID: 1, Nombre: documento1.pdf, Tipo: documento, Ruta: /ruta/a/documento1.pdf
Tarea ID: 1, Nombre: hello!.jpg, Tipo: imagen hello!, Ruta: C:Users/Fernando/Downloads/hello!.jpg
Tarea ID: 1, Nombre: Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf, Tipo: Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW, Ruta: C:/Users/Fernando/Downloads/Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf
Tarea ID: 1, Nombre: imagen1.jpg, Tipo: imagen, Ruta: /ruta/a/imagen1.jpg


In [6]:
agregar_archivo(1, "documento1.pdf", "documento", "/ruta/a/documento1.pdf")

Conexión a MongoDB Atlas exitosa!
Archivo 'documento1.pdf' agregado con éxito.


In [17]:
consultar_archivos(8)

Conexión a MongoDB Atlas exitosa!
Consultando archivos para la tarea ID: 8


In [8]:
agregar_archivo(1, "hello!.jpg", "imagen hello!", "C:Users/Fernando/Downloads/hello!.jpg")
agregar_archivo(1, "Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf", "Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW", "C:/Users/Fernando/Downloads/Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf")


Conexión a MongoDB Atlas exitosa!
Archivo 'hello!.jpg' agregado con éxito.
Conexión a MongoDB Atlas exitosa!
Archivo 'Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf' agregado con éxito.


In [9]:
consultar_archivos(1)  # Consulta los archivos asociados con la tarea 1
consultar_archivos(2)  # Consulta los archivos asociados con la tarea 2

Conexión a MongoDB Atlas exitosa!
Consultando archivos para la tarea ID: 1
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2AF7E250>
Tarea ID: 1, Nombre: imagen1.jpg, Tipo: imagen, Ruta: /ruta/a/imagen1.jpg
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2B062120>
Tarea ID: 1, Nombre: documento1.pdf, Tipo: documento, Ruta: /ruta/a/documento1.pdf
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2B0619A0>
Tarea ID: 1, Nombre: hello!.jpg, Tipo: imagen hello!, Ruta: C:Users/Fernando/Downloads/hello!.jpg
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2B0535B0>
Tarea ID: 1, Nombre: Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf, Tipo: Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW, Ruta: C:/Users/Fernando/Downloads/Terminos-y-Condiciones-Concurso-Capital-Semilla-MAR-2025-LOW.pdf
Archivo encontrado: <mongo_models.Archivo object at 0x0000021F2B0526D0>
Tarea ID: 1, Nombre: imagen1.jpg, Tipo: imagen, Ruta: /r

In [11]:
agregar_archivo(2, "Taller de Análisis.pdf", "Taller de Análisis", "C:/Users/Fernando/Downloads/Taller de Análisis.pdf")

Conexión a MongoDB Atlas exitosa!
Archivo 'Taller de Análisis.pdf' agregado con éxito.
